## 3: AgentCore Gateway

## Overview

This lab demonstrates how to create an AgentCore Gateway to expose Lambda-based tools through a standardized MCP (Model Context Protocol) interface. The Gateway provides a centralized, secure entry point for agents to discover and invoke tools.

## What You'll Learn

- Create an AgentCore Gateway with OAuth authentication
- Define tool specifications using JSON Schema
- Connect Lambda functions as gateway targets
- Test gateway access with different OAuth scopes
- (Optional) Implement fine-grained authorization with Cedar policies

## Prerequisites

- Completed 1 (Setup and Dependencies)
- Completed 2 (AgentCore Identity) - Cognito resources must be created
- Lambda functions deployed via CloudFormation

## Import Libraries and Initialize Configuration

Import necessary libraries and retrieve configuration from CloudFormation exports and SSM Parameter Store.

In [ ]:
# Import required libraries for AWS services, agent framework, and utilities
import os
import json
import boto3
import time
from strands import Agent
from strands.models import BedrockModel

# Import utility functions for AWS resource management
from utils.gateway_utils import (
    get_param_value,          # Retrieve SSM parameters
    put_ssm_parameter,        # Store SSM parameters
    get_cfn_export,           # Get CloudFormation exports
    get_role_arn              # Get IAM role ARNs
)

# Initialize AWS session and get account details
session = boto3.Session()
sts = session.client('sts')
identity = sts.get_caller_identity()
account_id = identity['Account']
region = boto3.Session().region_name or 'us-west-2'

print(f"Account ID: {account_id}")
print(f"Region: {region}")

# Load Cognito configuration from 2
print("\n📦 Loading Cognito configuration from 2...")
try:
    with open('cognito_config.json', 'r') as f:
        cognito_config = json.load(f)
    cognito_discovery_url = cognito_config['discovery_url']
    cognito_client_id = cognito_config['client_id']
    print("✓ Cognito configuration loaded from cognito_config.json")
except FileNotFoundError:
    raise Exception("❌ cognito_config.json not found. Please complete 2 first.")

# Retrieve CloudFormation exports for Lambda functions
print("\n📦 Retrieving Lambda function ARNs from CloudFormation...")
lambda_create_arn = get_cfn_export("lambda-CreateRefundRequestLambdaArn", region)
lambda_list_arn = get_cfn_export("lambda-ListReturnRequestLambdaArn", region)
lambda_approve_arn = get_cfn_export("lambda-ApproveReturnRequestLambdaArn", region)

print(f"✓ Cognito Discovery URL: {cognito_discovery_url}")
print(f"✓ Cognito Client ID: {cognito_client_id}")
print(f"✓ Lambda ARNs retrieved")
print("\n✅ Configuration loaded successfully!")

## Define Tool Specifications

Define JSON Schema specifications for each Lambda tool: CreateRefundRequest, ListReturnRequest, and ApproveReturnRequest.

In [ ]:
# Define tool specification for creating refund requests
# This tool allows customers to initiate a refund for their orders
create_tool_spec = [
  {
    "name":"create_refund_request",
    "description": "Create a new refund request for an order",
    "inputSchema": {
      "properties": {
        "amount": {
          "description": "The refund amount",
          "type": "number"
        },
        "order_id": {
          "description": "The order ID for which refund is requested",
          "type": "string"
        },
        "reason": {
          "description": "Reason for requesting the refund",
          "type": "string"
        },
        "user_id": {
          "description": "User ID of the requester",
          "type": "string"
        }
      },
      "required": [
        "order_id",
        "amount",
        "reason",
        "user_id"
      ],
      "type": "object"
    }
  }
]

# Define tool specification for listing refund requests
# This tool allows customers to view their refund request history
list_tool_spec = [
  {
    "name":"list_refund_requests",
    "description": "List all refund requests for a user",
    "inputSchema": {
      "properties": {
        "user_id": {
          "description": "User ID to list requests for",
          "type": "string"
        }
      },
      "required": [
        "user_id"
      ],
      "type": "object"
    }
  }
]

# Define tool specification for approving refund requests
# This is a SENSITIVE operation - only managers with write scope can access
approve_tool_spec = [
  {
    "name": "approve_refund_request",
    "description": "Approve or reject a refund request",
    "inputSchema": {
      "properties": {
        "approver_notes": {
          "description": "Notes from the approver",
          "type": "string"
        },
        "refund_request_id": {
          "description": "The refund request ID to approve/reject",
          "type": "string"
        },
        "status": {
          "description": "New status: approved or rejected",
          "type": "string"
        },
        "user_id": {
          "description": "User ID associated with the request",
          "type": "string"
        }
      },
      "required": [
        "refund_request_id",
        "user_id",
        "status"
      ],
      "type": "object"
    }
  }
]

## Create Gateway Execution Role

Create an IAM role that allows the gateway to invoke Lambda functions.

In [ ]:
import json

iam_client = boto3.client('iam')

# Define trust policy for AgentCore Gateway
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock-agentcore.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Define permissions policy for Lambda invocation
permissions_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "lambda:InvokeFunction"
            ],
            "Resource": [
                f"arn:aws:lambda:{region}:{account_id}:function:*RefundRequest*",
                f"arn:aws:lambda:{region}:{account_id}:function:*ReturnRequest*"
            ]
        }
    ]
}

role_name = "RefundManagementGatewayExecutionRole"

try:
    # Try to create the role
    print(f"🔧 Creating IAM role: {role_name}...")
    create_role_response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description="Execution role for AgentCore Gateway to invoke Lambda functions",
        Tags=[
            {'Key': 'Purpose', 'Value': 'AgentCoreGateway'},
            {'Key': 'Application', 'Value': 'RefundManagement'}
        ]
    )
    gateway_role_arn = create_role_response['Role']['Arn']
    print(f"✅ Role created: {gateway_role_arn}")
    
    # Attach inline policy for Lambda invocation
    iam_client.put_role_policy(
        RoleName=role_name,
        PolicyName="LambdaInvocationPolicy",
        PolicyDocument=json.dumps(permissions_policy)
    )
    print("✅ Lambda invocation policy attached")
    
    # Wait a few seconds for IAM to propagate
    print("⏳ Waiting for IAM role to propagate...")
    time.sleep(10)
    
except iam_client.exceptions.EntityAlreadyExistsException:
    # Role already exists, get its ARN
    print(f"ℹ️ Role {role_name} already exists, retrieving ARN...")
    get_role_response = iam_client.get_role(RoleName=role_name)
    gateway_role_arn = get_role_response['Role']['Arn']
    print(f"✅ Using existing role: {gateway_role_arn}")
    
    # Update the inline policy in case it changed
    try:
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName="LambdaInvocationPolicy",
            PolicyDocument=json.dumps(permissions_policy)
        )
        print("✅ Lambda invocation policy updated")
    except Exception as e:
        print(f"⚠️ Could not update policy: {e}")

print(f"\n✅ Gateway execution role ready: {gateway_role_arn}")

## Create AgentCore Gateway

Create the gateway with OAuth authentication (Cognito), MCP protocol, and IAM role for Lambda invocation.

In [ ]:
# Gateway configuration
# The gateway name must be unique within your AWS account
gateway_name = "RefundAgentGateway"
gateway_description = "Gateway for Refund Agent Tools"

# Auth configuration using pre-loaded Cognito parameters
# This tells the gateway how to validate OAuth tokens
auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [cognito_client_id],  # Only tokens from this client are accepted
        "discoveryUrl": cognito_discovery_url,   # Where to find Cognito's public keys
    }
}

print("✅ Gateway configuration prepared")
print(f"   Gateway Name: {gateway_name}")
print(f"   Allowed Client: {cognito_client_id}")

In [ ]:
# Initialize the AgentCore control plane client
gateway_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=region,
)

print("🔧 Creating AgentCore Gateway...")

try:
    # Attempt to create a new gateway
    create_response = gateway_client.create_gateway(
        name=gateway_name,
        roleArn=gateway_role_arn,  # IAM role for gateway to assume
        protocolType="MCP",         # Model Context Protocol
        authorizerType="CUSTOM_JWT", # Use Cognito JWT tokens
        authorizerConfiguration=auth_config,
        description=gateway_description,
    )

    gateway_id = create_response["gatewayId"]

    # Store gateway information for easy access
    gateway = {
        "id": gateway_id,
        "name": gateway_name,
        "gateway_url": create_response["gatewayUrl"],
        "gateway_arn": create_response["gatewayArn"],
    }
    
    # Save gateway details to JSON file for future use
    with open('gateway_config.json', 'w') as f:
        json.dump(gateway, f, indent=2)
    
    print(f"✅ Gateway created successfully!")
    print(f"   Gateway ID: {gateway_id}")
    print(f"   Gateway URL: {create_response['gatewayUrl']}")
    print(f"   Configuration saved to gateway_config.json")
    
except Exception as e:
    # Gateway already exists - retrieve from JSON file
    print(f"Gateway already exists, retrieving from gateway_config.json...")
    try:
        with open('gateway_config.json', 'r') as f:
            gateway = json.load(f)
        gateway_id = gateway["id"]
        print(f"✓ Using existing gateway: {gateway_id}")
    except FileNotFoundError:
        print("❌ gateway_config.json not found. Please check if gateway was created.")
        raise

## Add Lambda Targets to Gateway

Attach Lambda functions as gateway targets with MCP protocol and tool schemas.

In [ ]:
print("\n🔧 Creating Lambda targets...")

# Target 1: CreateRefundRequest
# This tool allows customers to initiate a refund for their orders
print("\n1️⃣ Creating CreateRefundRequest target...")
try:
    lambda_target_config = {
        "mcp": {
            "lambda": {
                "lambdaArn": lambda_create_arn,
                "toolSchema": {"inlinePayload": create_tool_spec},
            }
        }
    }
    
    # Use gateway's IAM role for Lambda invocation
    credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]

    create_target_response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name="CreateRefundRequest",
        description="Lambda function for creating a new refund request",
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=credential_config,
    )
    print(f"   ✅ Target created: CreateRefundRequest ({create_target_response['targetId']})")
except Exception as e:
    print(f"   ⚠️ Error: {str(e)}")

# Target 2: ListReturnRequest
# This tool allows customers to view their refund request history
print("\n2️⃣ Creating ListReturnRequest target...")
try:
    lambda_target_config = {
        "mcp": {
            "lambda": {
                "lambdaArn": lambda_list_arn,
                "toolSchema": {"inlinePayload": list_tool_spec},
            }
        }
    }
    
    credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]

    create_target_response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name="ListReturnRequest",
        description="Lambda function for listing all refund requests",
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=credential_config,
    )
    print(f"   ✅ Target created: ListReturnRequest ({create_target_response['targetId']})")
except Exception as e:
    print(f"   ⚠️ Error: {str(e)}")

# Target 3: ApproveReturnRequest
# This is a SENSITIVE operation - only managers with write scope can access
print("\n3️⃣ Creating ApproveReturnRequest target...")
try:
    lambda_target_config = {
        "mcp": {
            "lambda": {
                "lambdaArn": lambda_approve_arn,
                "toolSchema": {"inlinePayload": approve_tool_spec},
            }
        }
    }
    
    credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]

    create_target_response = gateway_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name="ApproveReturnRequest",
        description="Lambda function for approving/rejecting a return request",
        targetConfiguration=lambda_target_config,
        credentialProviderConfigurations=credential_config,
    )
    print(f"   ✅ Target created: ApproveReturnRequest ({create_target_response['targetId']})")
except Exception as e:
    print(f"   ⚠️ Error: {str(e)}")

print("\n✅ All Lambda targets configured")

## Test Gateway Access

Verify the gateway is working by listing available tools with a valid OAuth token.

In [ ]:
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
from utils.identity_ssm_utils import reauthenticate_user

# Get bearer token using Cognito credentials from 2
token = reauthenticate_user(cognito_client_id, cognito_config['client_secret'])

# Set up MCP client
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway["gateway_url"],
        headers={"Authorization": f"Bearer {token}"},
    )
)
with mcp_client:
    for tool in mcp_client.list_tools_sync():
        print(f"Tool name {tool.mcp_tool.name}")
        print("Tool Description ", tool.mcp_tool.description)

### Summary

You've created an AgentCore Gateway with OAuth authentication, defined tool specifications, and connected Lambda targets.

### Next Steps

- **4: AgentCore Memory** - Explore memory strategies